<h1>INTRODUCTION</h1>

Toronto and New York are places that diverse in many ways. Both are multicultural as well as the financial hubs of their respective countries. We want to explore how much they are similar or dissimilar in aspects from a tourist point of view regarding food, accommodation, beautiful places, and many more.
     

<h2>DATA DESCRIPTION</h2>

For this problem, we will get the services of Foursquare API to explore the data of two cities, in terms of their neighborhoods. The data also include the information about the places around each neighborhood like restaurants, hotels, coffee shops, parks, theaters, art galleries, museums and many more. 
<br>We selected one Borough from each city to analyze their neighborhoods. Manhattan from New York and Downtown Toronto from Toronto. 
<br>We will use machine learning technique, “Clustering” to segment the neighborhoods with similar objects on the basis of each neighborhood data. These objects will be given priority on the basis of foot traffic (activity) in their respective neighborhoods. This will help to locate the tourist’s areas and hubs, and then we can judge the similarity or dissimilarity between two cities on that basis.
<br>
<br>For Downtown Toronto case, we have extracted table of Toronto’s Borough from Wikipedia page. Then we arrange the data according to our requirements. In the arrangement phase, which applied multiple steps including but not limited to, eliminating “Not assigned” values, combine neighborhoods which have same geographical coordinates at each borough and sorted against the concerned borough. For data verification and further exploration, we use Foursquare API to get the coordinates of Downtown Toronto and explore its neighborhoods. The neighborhoods are further characterized as venues and venue categories.
For Manhattan, we used a saved data file which is already explored through foursquare API in which we have extracted all the boroughs of New York and then sorted against the concerned borough. Then we explored the Manhattan neighborhoods as venues and venue categories.

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Visualization
import matplotlib.pyplot
import seaborn as sns
# Too see full dataframe...
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


In [8]:
# Link To Extract
path='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# Read File
df_wiki=pd.read_html(path)
#Check the type
type(df_wiki)
# Call the position where the table is stored
neighborhood=df_wiki[0]
# Rename the Columns
neighborhood.rename(columns={0:'Postal Code', 1: 'Borough', 2: 'Neighborhood'}, inplace=True)
# Eliminate the first row
neighborhood=neighborhood.drop([0])
neighborhood

,Postal Code,Borough,Neighborhood
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
10,M2B,Not assigned,NaN


In [9]:
# Eliminate "Not assigned", categorical values from "Borough" Column
neighborhood=neighborhood[neighborhood.Borough !='Not assigned']
# Making DataFrame
neighborhood=pd.DataFrame(neighborhood)
# Merging rows with same Postcode
neighborhood.set_index(['Postal Code','Borough'],inplace=True)
merge_result = neighborhood.groupby(level=['Postal Code','Borough'], sort=False).agg( ','.join)


In [10]:
# Setting the index
serial_wise=merge_result.reset_index()
# Assign the 'Borough' column value to 'Neighborhood' where 'Not assigned' occurs
serial_wise.loc[4, 'Neighborhood']='Queen\'s Park'
# Saving the file for future use!
serial_wise.to_excel('wikipedia_table.xls')
# Showing the Data Frame
df=pd.DataFrame(serial_wise)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,Queen's Park


In [16]:
# Geographical Coordinates
df1=pd.read_csv("http://cocl.us/Geospatial_data")
#Cancatenation
frames=[df,df1]
frames=pd.concat(frames, axis=1, sort=False)
# Merging the two columns on 'Postcode'
merge_columns=pd.merge(df, df1, left_on='Postal Code', right_on='Postal Code')
# Save the Data Frame
merge_columns.to_csv('neigbors_geographical.csv')
merge_columns.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [18]:
# Sorting
# set index for only Downtown Toronto
downtown_toronto_data = merge_columns[merge_columns['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
# eliminate 'Postcode' column
downtown_toronto_data=downtown_toronto_data.drop(['Postal Code'], axis=1)
downtown_toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,Downtown Toronto,Berczy Park,43.644771,-79.373306


<h3>Select "Manhattan" from New York Boroughs</h3>

In [28]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
import json
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    

In [32]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [34]:
#Transform the data into pandas dataframe
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
# And make sure that the dataset has all 5 boroughs and 306 neighborhoods.
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

neighborhoods.head()

The dataframe has 5 boroughs and 306 neighborhoods.


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [35]:
# Creating new Dataframe manhattan_data
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


<h3>Foursquare API</h3>

In [36]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'Z5TR1K2GXOGC2K5L4IYOT53TEKYOINTCS1XTISVHYIMHIGUQ' 
CLIENT_SECRET = 'F4JKWQYW3RT1J511Y1CXCQQSZSGVKW0UA3BBUGUENTDBFQYX' 
VERSION = '20180604'
limit = 20
print('Your credentails:')
print('CLIENT_ID:'+ CLIENT_ID)
print('CLIENT_SECRET:'+ CLIENT_SECRET)

Your credentails:
CLIENT_ID:Z5TR1K2GXOGC2K5L4IYOT53TEKYOINTCS1XTISVHYIMHIGUQ
CLIENT_SECRET:F4JKWQYW3RT1J511Y1CXCQQSZSGVKW0UA3BBUGUENTDBFQYX


In [39]:
# get the geographical coordinates of Downtown Toronto
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim(user_agent="me")
location = geolocator.geocode(address)
latitude_downtown_toronto = location.latitude
longitude_downtown_toronto = location.longitude
print("Downtown Toronto","latitude",latitude_downtown_toronto, "& " "longitude" ,longitude_downtown_toronto)

Downtown Toronto latitude 43.6563221 & longitude -79.3809161


In [40]:
# get the geographical coordinates of Manhattan.
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="me")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


<h3>VISUALIZATION</h3>

<h4>Downtown Toronto - Before clustering</h4>

In [41]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

In [42]:
from folium import plugins
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)
# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(map_downtown_toronto)
# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(incidents)  
    
map_downtown_toronto

<h4>Manhattan</h4>

In [43]:
# let's visualizat Manhattan the neighborhoods in it.
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [44]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

grouping = plugins.MarkerCluster().add_to(map_manhattan)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(grouping)  
    
map_manhattan

<h3>ANALYSIS</h3>
<br>We analyze both boroughs neighborhoods through one hot encoding (giving ‘1’ if a venue category is there, and ‘0’ in case of venue category is not there)

#### Exploring Neighborhoods in Downtown Toronto

In [45]:
# Let's create a function to repeat the process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
# Write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.
downtown_toronto_venues = getNearbyVenues(names=downtown_toronto_data['Neighborhood'],
                                   latitudes=downtown_toronto_data['Latitude'],
                                   longitudes=downtown_toronto_data['Longitude'],)                                

Regent Park, Harbourfront
Queen's Park
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [47]:
# check the size of the resulting dataframe
print(downtown_toronto_venues.shape)
downtown_toronto_venues.head()

(357, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [48]:
# Let's check how many venues were returned for each neighborhood
downtown_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,20,20,20,20,20,20
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,20,20,20,20,20,20
Christie,17,17,17,17,17,17
Church and Wellesley,20,20,20,20,20,20
"Commerce Court, Victoria Hotel",20,20,20,20,20,20
"First Canadian Place, Underground city",20,20,20,20,20,20
"Garden District, Ryerson",20,20,20,20,20,20
"Harbourfront East, Union Station, Toronto Islands",20,20,20,20,20,20


In [49]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 125 uniques categories.


##### Analyzing Each Neighborhood

In [50]:
# one hot encoding
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_toronto_onehot['Neighborhood'] = downtown_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]

downtown_toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bar,Basketball Stadium,Beer Bar,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Distribution Center,Electronics Store,Farmers Market,Fish Market,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gastropub,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Martial Arts Dojo,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [51]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [52]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in downtown_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0  Seafood Restaurant  0.10
1        Cocktail Bar  0.05
2  Basketball Stadium  0.05
3         Coffee Shop  0.05
4          Restaurant  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4             Plane  0.06


----Central Bay Street----
                        venue  freq
0                 Coffee Shop  0.30
1   Middle Eastern Restaurant  0.05
2                        Park  0.05
3                         Spa  0.05
4  Modern European Restaurant  0.05


----Christie----
           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12
3      Nightclub  0.06
4     Baby Store  0.06


----Church and Wellesley----
             venue  freq
0      Pizza Place  0.05
1  Bubble Tea Shop  0.05
2     Burger Joint  0.05
3     

In [53]:
# Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [54]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_toronto_grouped['Neighborhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Basketball Stadium,Bakery,Park,Cocktail Bar,Coffee Shop,Museum,Breakfast Spot,Concert Hall,Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Plane,Bar,Boat or Ferry,Boutique,Rental Car Location,Sculpture Garden,Harbor / Marina
2,Central Bay Street,Coffee Shop,Café,Sushi Restaurant,Modern European Restaurant,Bubble Tea Shop,Ramen Restaurant,Middle Eastern Restaurant,Sandwich Place,Spa,Japanese Restaurant
3,Christie,Grocery Store,Café,Park,Italian Restaurant,Candy Store,Coffee Shop,Nightclub,Restaurant,Baby Store,Athletics & Sports
4,Church and Wellesley,Pizza Place,Park,Bookstore,Restaurant,Ramen Restaurant,Breakfast Spot,Bubble Tea Shop,Burger Joint,Mexican Restaurant,Beer Bar
5,"Commerce Court, Victoria Hotel",Café,Coffee Shop,Bakery,Gym,Gastropub,Ice Cream Shop,Japanese Restaurant,Deli / Bodega,Museum,Pub
6,"First Canadian Place, Underground city",Café,Coffee Shop,Restaurant,Gym / Fitness Center,Bookstore,Deli / Bodega,Bakery,Steakhouse,Pub,Seafood Restaurant
7,"Garden District, Ryerson",Café,Mexican Restaurant,Shopping Mall,Restaurant,Ramen Restaurant,Coffee Shop,Plaza,Steakhouse,Electronics Store,Sandwich Place
8,"Harbourfront East, Union Station, Toronto Islands",Park,Plaza,Hotel,Café,Supermarket,Performing Arts Venue,New American Restaurant,Bubble Tea Shop,Salad Place,Lake
9,"Kensington Market, Chinatown, Grange Park",Café,Vietnamese Restaurant,Mexican Restaurant,Caribbean Restaurant,Bakery,Wine Bar,Fish Market,Dessert Shop,Coffee Shop,Organic Grocery


##### Clustering Neighborhoods

In [55]:
# set number of clusters
kclusters = 5

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 4, 3, 2, 0, 0, 0, 2, 0], dtype=int32)

In [56]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
downtown_toronto_merged = downtown_toronto_data

# add clustering labels
downtown_toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Breakfast Spot,Farmers Market,Chocolate Shop,Pub,Restaurant,Performing Arts Venue,Dessert Shop,Bakery
1,Downtown Toronto,Queen's Park,43.662301,-79.389494,2,Coffee Shop,Sushi Restaurant,Wings Joint,Park,Arts & Crafts Store,Beer Bar,Burrito Place,Creperie,Diner,Distribution Center
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Café,Mexican Restaurant,Shopping Mall,Restaurant,Ramen Restaurant,Coffee Shop,Plaza,Steakhouse,Electronics Store,Sandwich Place
3,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Gastropub,Coffee Shop,Café,Creperie,Hotel,Italian Restaurant,BBQ Joint,Cosmetics Shop,Food Truck,Restaurant
4,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Seafood Restaurant,Basketball Stadium,Bakery,Park,Cocktail Bar,Coffee Shop,Museum,Breakfast Spot,Concert Hall,Restaurant


In [57]:
# create map
map_clusters = folium.Map(location=[latitude_downtown_toronto, longitude_downtown_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighborhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##### Examine Clusters

Cluster 1 (Airport Lounge, Coffee Shop, Cafe, Restaurants & Grocery Store)

In [58]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Bay Street,Coffee Shop,Café,Sushi Restaurant,Modern European Restaurant,Bubble Tea Shop,Ramen Restaurant,Middle Eastern Restaurant,Sandwich Place,Spa,Japanese Restaurant
6,Christie,Grocery Store,Café,Park,Italian Restaurant,Candy Store,Coffee Shop,Nightclub,Restaurant,Baby Store,Athletics & Sports
7,"Richmond, Adelaide, King",Coffee Shop,Seafood Restaurant,Gym / Fitness Center,Steakhouse,Hotel,Concert Hall,Lounge,Opera House,Café,Pizza Place
9,"Toronto Dominion Centre, Design Exchange",Café,Coffee Shop,Gym / Fitness Center,Pub,Beer Bar,Deli / Bodega,Restaurant,Bakery,Bookstore,Japanese Restaurant
14,Rosedale,Park,Playground,Trail,Deli / Bodega,Cheese Shop,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym
15,Stn A PO Boxes,Café,Cocktail Bar,Tailor Shop,Museum,Comfort Food Restaurant,Restaurant,Concert Hall,Beer Bar,Park,Farmers Market
17,"First Canadian Place, Underground city",Café,Coffee Shop,Restaurant,Gym / Fitness Center,Bookstore,Deli / Bodega,Bakery,Steakhouse,Pub,Seafood Restaurant
18,Church and Wellesley,Pizza Place,Park,Bookstore,Restaurant,Ramen Restaurant,Breakfast Spot,Bubble Tea Shop,Burger Joint,Mexican Restaurant,Beer Bar


Cluster 2 (Gastropubs)

In [59]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Plane,Bar,Boat or Ferry,Boutique,Rental Car Location,Sculpture Garden,Harbor / Marina


Cluster 3 (Cafes)

In [60]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Park,Breakfast Spot,Farmers Market,Chocolate Shop,Pub,Restaurant,Performing Arts Venue,Dessert Shop,Bakery
1,Queen's Park,Coffee Shop,Sushi Restaurant,Wings Joint,Park,Arts & Crafts Store,Beer Bar,Burrito Place,Creperie,Diner,Distribution Center
4,Berczy Park,Seafood Restaurant,Basketball Stadium,Bakery,Park,Cocktail Bar,Coffee Shop,Museum,Breakfast Spot,Concert Hall,Restaurant
8,"Harbourfront East, Union Station, Toronto Islands",Park,Plaza,Hotel,Café,Supermarket,Performing Arts Venue,New American Restaurant,Bubble Tea Shop,Salad Place,Lake
12,"Kensington Market, Chinatown, Grange Park",Café,Vietnamese Restaurant,Mexican Restaurant,Caribbean Restaurant,Bakery,Wine Bar,Fish Market,Dessert Shop,Coffee Shop,Organic Grocery
16,"St. James Town, Cabbagetown",Restaurant,Café,Gift Shop,Indian Restaurant,Deli / Bodega,Jewelry Store,Diner,Bakery,Japanese Restaurant,Italian Restaurant


Cluster 4 (Coffee Shop, Cafe, Park & Japanese Restaurant)

In [61]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,St. James Town,Gastropub,Coffee Shop,Café,Creperie,Hotel,Italian Restaurant,BBQ Joint,Cosmetics Shop,Food Truck,Restaurant


Cluster 5 (Seafood, steakhouse, Hotel & Cafe)

In [62]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Garden District, Ryerson",Café,Mexican Restaurant,Shopping Mall,Restaurant,Ramen Restaurant,Coffee Shop,Plaza,Steakhouse,Electronics Store,Sandwich Place
10,"Commerce Court, Victoria Hotel",Café,Coffee Shop,Bakery,Gym,Gastropub,Ice Cream Shop,Japanese Restaurant,Deli / Bodega,Museum,Pub
11,"University of Toronto, Harbord",Restaurant,Bookstore,Bakery,Japanese Restaurant,Yoga Studio,Italian Restaurant,Café,College Gym,Comfort Food Restaurant,Sandwich Place


### Exploring Neighborhoods in Manhattan

In [63]:
# Let's create a function to repeat the same process to all the neighborhoods in Manhattan
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [64]:
# Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude'],
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [65]:
# Let's check how many venues were returned for each neighborhood
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,20,20,20,20,20,20
Carnegie Hill,20,20,20,20,20,20
Central Harlem,20,20,20,20,20,20
Chelsea,20,20,20,20,20,20
Chinatown,20,20,20,20,20,20
Civic Center,20,20,20,20,20,20
Clinton,20,20,20,20,20,20
East Harlem,20,20,20,20,20,20
East Village,20,20,20,20,20,20


In [66]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 217 uniques categories.


##### Analyzing the Neighborhoods

In [67]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Bar,Beer Garden,Beer Store,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Heliport,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,New American Restaurant,Newsstand,Noodle House,Office,Opera House,Optical Shop,Outdoor Sculpture,Outdoors & Recreation,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pub,Public Art,Ramen Restaurant,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Taco Place,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Tiki Bar,Tourist Information Center,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [68]:
# Set Index
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()


In [69]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
            venue  freq
0   Memorial Site  0.15
1            Park  0.15
2      Food Court  0.10
3           Plaza  0.05
4  Sandwich Place  0.05


----Carnegie Hill----
                  venue  freq
0    Italian Restaurant  0.10
1                   Gym  0.10
2  Gym / Fitness Center  0.10
3          Dance Studio  0.05
4            Bagel Shop  0.05


----Central Harlem----
                venue  freq
0  African Restaurant  0.10
1   French Restaurant  0.10
2           Juice Bar  0.05
3                 Bar  0.05
4        Dessert Shop  0.05


----Chelsea----
                venue  freq
0      Ice Cream Shop  0.10
1  Seafood Restaurant  0.10
2             Theater  0.10
3            Beer Bar  0.05
4              Market  0.05


----Chinatown----
                 venue  freq
0       Sandwich Place  0.10
1   Chinese Restaurant  0.10
2  Indie Movie Theater  0.05
3               Bakery  0.05
4          Pizza Place  0.05


----Civic Center----
                             v

In [70]:
# Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [71]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Memorial Site,Park,Food Court,Plaza,Gym,Smoke Shop,Shopping Mall,Scenic Lookout,Sandwich Place,Monument / Landmark
1,Carnegie Hill,Gym,Gym / Fitness Center,Italian Restaurant,Dance Studio,Café,Gourmet Shop,Coffee Shop,Bar,Bagel Shop,Bookstore
2,Central Harlem,French Restaurant,African Restaurant,Juice Bar,Music Venue,Boutique,Cocktail Bar,Ethiopian Restaurant,Library,Beer Bar,Gym / Fitness Center
3,Chelsea,Theater,Ice Cream Shop,Seafood Restaurant,Speakeasy,Café,Market,Scenic Lookout,Chinese Restaurant,Beer Bar,Taco Place
4,Chinatown,Sandwich Place,Chinese Restaurant,Indie Movie Theater,Tea Room,Hotpot Restaurant,Ice Cream Shop,Greek Restaurant,Museum,English Restaurant,New American Restaurant
5,Civic Center,Spa,French Restaurant,Yoga Studio,Gym,Molecular Gastronomy Restaurant,Burrito Place,Falafel Restaurant,Martial Arts Dojo,Furniture / Home Store,Sushi Restaurant
6,Clinton,Theater,Gym / Fitness Center,Hotel,Pie Shop,Food Court,Sporting Goods Shop,Sports Bar,Café,Mediterranean Restaurant,Lounge
7,East Harlem,Mexican Restaurant,Latin American Restaurant,Thai Restaurant,Pharmacy,Steakhouse,Beer Bar,Sandwich Place,Bakery,Street Art,Cocktail Bar
8,East Village,Vietnamese Restaurant,Dessert Shop,Cheese Shop,Japanese Restaurant,Scandinavian Restaurant,Swiss Restaurant,Coffee Shop,Bar,Juice Bar,Bagel Shop
9,Financial District,Coffee Shop,Gym / Fitness Center,Jewelry Store,Falafel Restaurant,Café,Steakhouse,Seafood Restaurant,New American Restaurant,Salad Place,Spa


##### Clustering Neighborhoods

In [72]:
# Run k-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 2, 1, 2, 1, 2, 3, 1, 1], dtype=int32)

In [73]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,3,Gym,Coffee Shop,Yoga Studio,Pizza Place,Department Store,Diner,Discount Store,Donut Shop,Sandwich Place,Seafood Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,1,Sandwich Place,Chinese Restaurant,Indie Movie Theater,Tea Room,Hotpot Restaurant,Ice Cream Shop,Greek Restaurant,Museum,English Restaurant,New American Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Wine Shop,Café,Park,Deli / Bodega,Ramen Restaurant,Breakfast Spot,Market,Liquor Store,Cocktail Bar,Coffee Shop
3,Manhattan,Inwood,40.867684,-73.921210,1,Wine Bar,Café,Park,Yoga Studio,Diner,Spanish Restaurant,Farmers Market,Frozen Yogurt Shop,Bistro,Latin American Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Yoga Studio,Cocktail Bar,Mexican Restaurant,Wine Bar,Cosmetics Shop,Smoke Shop,Café,Caribbean Restaurant,Mediterranean Restaurant,Coffee Shop


In [74]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##### Examine Clusters

Cluster 1 Residencial

In [75]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,West Village,Cocktail Bar,Coffee Shop,Bakery,Italian Restaurant,Food & Drink Shop,Boutique,Gourmet Shop,Board Shop,Hardware Store,Austrian Restaurant


Cluster 2 Commercial Places

In [76]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Sandwich Place,Chinese Restaurant,Indie Movie Theater,Tea Room,Hotpot Restaurant,Ice Cream Shop,Greek Restaurant,Museum,English Restaurant,New American Restaurant
3,Inwood,Wine Bar,Café,Park,Yoga Studio,Diner,Spanish Restaurant,Farmers Market,Frozen Yogurt Shop,Bistro,Latin American Restaurant
5,Manhattanville,Italian Restaurant,Café,BBQ Joint,Bike Trail,Sushi Restaurant,Gastropub,Coffee Shop,Climbing Gym,Lounge,Juice Bar
8,Upper East Side,Hotel,Jazz Club,Sandwich Place,Bookstore,Coffee Shop,Boutique,Seafood Restaurant,Bar,Optical Shop,Bakery
9,Yorkville,Wine Shop,Deli / Bodega,Gym,Liquor Store,Beer Store,Coffee Shop,Monument / Landmark,Bagel Shop,Gym / Fitness Center,Dog Run
11,Roosevelt Island,Playground,Japanese Restaurant,Farmers Market,Bus Line,Bubble Tea Shop,School,Liquor Store,Sandwich Place,Kosher Restaurant,Residential Building (Apartment / Condo)
12,Upper West Side,Bakery,Italian Restaurant,American Restaurant,Juice Bar,Greek Restaurant,Bagel Shop,Lingerie Store,Chinese Restaurant,Tiki Bar,Bookstore
20,Lower East Side,Cocktail Bar,Art Gallery,Italian Restaurant,Coffee Shop,Mediterranean Restaurant,Juice Bar,Chinese Restaurant,Filipino Restaurant,Japanese Restaurant,Yoga Studio
21,Tribeca,Park,Yoga Studio,Sushi Restaurant,Hotel,Indie Theater,Italian Restaurant,Greek Restaurant,Men's Store,Cycle Studio,Poke Place
22,Little Italy,Bakery,History Museum,Thai Restaurant,Pilates Studio,Coffee Shop,Noodle House,Newsstand,Chocolate Shop,Salad Place,Chinese Restaurant


Cluster 3 Tourist Areas and hubs

In [77]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Wine Shop,Café,Park,Deli / Bodega,Ramen Restaurant,Breakfast Spot,Market,Liquor Store,Cocktail Bar,Coffee Shop
4,Hamilton Heights,Yoga Studio,Cocktail Bar,Mexican Restaurant,Wine Bar,Cosmetics Shop,Smoke Shop,Café,Caribbean Restaurant,Mediterranean Restaurant,Coffee Shop
6,Central Harlem,French Restaurant,African Restaurant,Juice Bar,Music Venue,Boutique,Cocktail Bar,Ethiopian Restaurant,Library,Beer Bar,Gym / Fitness Center
13,Lincoln Square,Theater,Concert Hall,Performing Arts Venue,Indie Movie Theater,Circus,Gift Shop,Fountain,College Arts Building,Plaza,Opera House
16,Murray Hill,Burger Joint,Hotel,Coffee Shop,Bagel Shop,Museum,Shanghai Restaurant,Event Space,Sandwich Place,Sushi Restaurant,Taco Place
18,Greenwich Village,Italian Restaurant,Yoga Studio,Coffee Shop,Café,Caribbean Restaurant,Seafood Restaurant,French Restaurant,New American Restaurant,Sandwich Place,Sushi Restaurant
19,East Village,Vietnamese Restaurant,Dessert Shop,Cheese Shop,Japanese Restaurant,Scandinavian Restaurant,Swiss Restaurant,Coffee Shop,Bar,Juice Bar,Bagel Shop
29,Financial District,Coffee Shop,Gym / Fitness Center,Jewelry Store,Falafel Restaurant,Café,Steakhouse,Seafood Restaurant,New American Restaurant,Salad Place,Spa
34,Sutton Place,Beer Garden,Grocery Store,Gym,Yoga Studio,Thai Restaurant,Bagel Shop,Beer Store,Furniture / Home Store,Greek Restaurant,Gym / Fitness Center


Cluster 4 Center Activity

In [78]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Gym,Coffee Shop,Yoga Studio,Pizza Place,Department Store,Diner,Discount Store,Donut Shop,Sandwich Place,Seafood Restaurant
7,East Harlem,Mexican Restaurant,Latin American Restaurant,Thai Restaurant,Pharmacy,Steakhouse,Beer Bar,Sandwich Place,Bakery,Street Art,Cocktail Bar
10,Lenox Hill,Thai Restaurant,Gym,Cycle Studio,Dessert Shop,Smoke Shop,French Restaurant,Chinese Restaurant,Liquor Store,Salad Place,Taco Place
14,Clinton,Theater,Gym / Fitness Center,Hotel,Pie Shop,Food Court,Sporting Goods Shop,Sports Bar,Café,Mediterranean Restaurant,Lounge
15,Midtown,Clothing Store,Hotel,Plaza,French Restaurant,Cycle Studio,Park,Salad Place,Smoke Shop,Spa,Food Truck
25,Manhattan Valley,Pizza Place,Bar,Yoga Studio,Park,Bubble Tea Shop,Mexican Restaurant,Fried Chicken Joint,Chinese Restaurant,Coffee Shop,Bakery
30,Carnegie Hill,Gym,Gym / Fitness Center,Italian Restaurant,Dance Studio,Café,Gourmet Shop,Coffee Shop,Bar,Bagel Shop,Bookstore
33,Midtown South,Korean Restaurant,Hotel,Japanese Restaurant,Coffee Shop,Building,Fried Chicken Joint,Bookstore,Scenic Lookout,Lingerie Store,Grocery Store
37,Stuyvesant Town,Boat or Ferry,Park,Gas Station,Bistro,Cocktail Bar,Baseball Field,Coffee Shop,Gym,Gym / Fitness Center,Harbor / Marina
39,Hudson Yards,American Restaurant,Gym / Fitness Center,Building,Furniture / Home Store,Cocktail Bar,Music School,Café,Theater,Residential Building (Apartment / Condo),Gym


Cluster 5 Cultural & Going Out Places

In [79]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Chelsea,Theater,Ice Cream Shop,Seafood Restaurant,Speakeasy,Café,Market,Scenic Lookout,Chinese Restaurant,Beer Bar,Taco Place


<h2>RESULTS</h2>
<br>
After clustering the data of the respective neighborhoods, both cities have venues which can be explored and attract the Tourists. The neighborhoods are much similar in features like Theaters, opera houses, food places, clubs, museums, parks etc. As far as concern to dissimilarity, it differs in terms of some unique places like historical places and monuments.


Observations & Recommendations
We recommend Downtown Toronto Neighborhoods will be considered first to visit. The tourists have an easily travelling access due to Airport facility, which not only saves time but also helps to save money. This saved money can be utilized to explore more, the attracting venues.
Conclusion
The downtown Toronto and Manhattan neighborhoods have more like similar venues. As we know that every place is unique in its own way, so that’s argument is present in both neighborhoods. The dissimilarity exists in terms of some different venues and facilities but not on a larger extent.